In [0]:
pip install azure-storage-file-datalake

In [0]:
import pandas as pd
from azure.storage.filedatalake import DataLakeServiceClient
from datetime import datetime

# Azure Storage connection details
account_name = "cdmo"
account_key = "XXXXXXXXXXXXXXXXXXXX"
container_name = "config"

# Data for the configuration file
data = [
    ["Bronze", "00-landing", "data/raw/customerfeedback/", "csv", "01-bronze", "customerfeedback/", "delta", "true"],
    ["Bronze", "00-landing", "data/raw/manufacturebatch/", "csv", "01-bronze", "manufacturebatch/", "delta", "true"],
    ["Bronze", "00-landing", "data/raw/productformula/", "csv", "01-bronze", "productformula/", "delta", "true"],
    ["Silver", "01-bronze", "customerfeedback/", "delta", "02-silver", "customerfeedback_transformed/", "delta", "true"],
    ["Silver", "01-bronze", "manufacturebatch/", "delta", "02-silver", "manufacturebatch_transformed/", "delta", "true"],
    ["Silver", "01-bronze", "productformula/", "delta", "02-silver", "productformula_transformed/", "delta", "true"],
]

# Column names for the configuration file
columns = [
    "Layer", "SourceContainer", "SourcePath", "SourceFormat",
    "TargetContainer", "TargetPath", "TargetFormat", "AddTimestamp"
]

# Create a DataFrame from the data
df = pd.DataFrame(data, columns=columns)

# Append the current date to the metadata config file name
current_date = datetime.now().strftime("%Y%m%d")
config_file_name = f"metadata_config_{current_date}.csv"

# Save the DataFrame to a CSV file locally
df.to_csv(config_file_name, index=False)
print(f"Configuration file '{config_file_name}' created successfully.")

# Upload the CSV file to the config directory in ADLS
def upload_to_adls(local_file_name, container_name):
    try:
        # Initialize the DataLakeServiceClient
        service_client = DataLakeServiceClient(
            account_url=f"https://{account_name}.dfs.core.windows.net",
            credential=account_key,
        )
        
        # Get the file system client for the container
        file_system_client = service_client.get_file_system_client(container_name)
        
        # Upload the file directly into the container without creating a subdirectory
        with open(local_file_name, "rb") as data:
            file_client = file_system_client.get_file_client(local_file_name)
            file_client.upload_data(data, overwrite=True)
        
        print(f"File '{local_file_name}' uploaded to ADLS in container '{container_name}'.")
    except Exception as e:
        print(f"Error uploading file to ADLS: {e}")

# Call the function to upload the file to ADLS
upload_to_adls(config_file_name, container_name)
